# Dataset Cleaning

*Image Deleter and Relabeler by [Zach Caceres](http://zachcaceres.com/now/) in collaboration with Jason Patnick, Duplicate Finder by [Francisco Ingham](https://medium.com/@fpingham)*

In this notebook we will show you how to take advantage of fastai widgets to clean your dataset! We will delete images that do not correspond, relabel images with incorrect labels and delete duplicates. For this, we will use the CIFAR10 dataset but you can use it in your own custom dataset you built with the [google images dataset](https://github.com/fpingham/google-images-dataset) notebook.

# Training your first model

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.vision import *

In [3]:
path = untar_data(URLs.CIFAR)

We will first train a model since it will suggest us which are the images that are most likely to be mislabelled or not belong to our dataset. We will also use the weights of the pretrained model to find similar images that might be duplicates.

In [4]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4).normalize(imagenet_stats)

You can deactivate this warning by passing `no_check=True`.


/home/jupyter/fastai/fastai/basic_data.py:202: UserWarning: There seems to be something wrong with your dataset, can't access self.train_ds[i] for all i in [30627, 20390, 26880, 37213, 14097, 22387, 45624, 40812, 31032, 3055, 34336, 45497, 37381, 4394, 9196, 31673, 16311, 20759, 18348, 30362, 44072, 10265, 23108, 26586, 25339, 24016, 40122, 20017, 47053, 13019, 25899, 23149, 37070, 18201, 5269, 5788, 1135, 9933, 28296, 7795, 43274, 34229, 2251, 34366, 7236, 40770, 10148, 47723, 32908, 19690, 44240, 47076, 44381, 4400, 29549, 34562, 13569, 20353, 4323, 33200, 9554, 30951, 1665, 41760]
  warn(f"There seems to be something wrong with your dataset, can't access self.train_ds[i] for all i in {idx}")


In [5]:
learn = create_cnn(data, models.resnet34, metrics=error_rate)

In [6]:
# learn.fit_one_cycle(4)

In [7]:
# learn.save('stage-1');

In [6]:
learn.load('stage-1');

## Cleaning your dataset

In [7]:
from fastai.widgets import *
import torch

We will create an `ImageDataBunch` with all the images in the training set since the widget will only use the images in the training set.

In [8]:
# We create a databunch with all the data in the training set and no validation set (DatasetFormatter uses only the training set)
db = (ImageItemList.from_folder(path)
                   .no_split()
                   .label_from_folder()
                   .databunch())

To start, we will sort the indices of our images by the highest loss 
images since this suggests that the image might be mislabeled or just not belong to the dataset.

In [24]:
learn_rel = create_cnn(db, models.resnet34, metrics=error_rate)
learn_rel.load('stage-1')

Learner(data=ImageDataBunch;

Train: LabelList
y: CategoryList (60000 items)
[Category dog, Category dog, Category dog, Category dog, Category dog]...
Path: /home/jupyter/.fastai/data/cifar10
x: ImageItemList (60000 items)
[Image (3, 32, 32), Image (3, 32, 32), Image (3, 32, 32), Image (3, 32, 32), Image (3, 32, 32)]...
Path: /home/jupyter/.fastai/data/cifar10;

Valid: LabelList
y: CategoryList (0 items)
[]...
Path: /home/jupyter/.fastai/data/cifar10
x: ImageItemList (0 items)
[]...
Path: /home/jupyter/.fastai/data/cifar10;

Test: None, model=Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)

In [15]:
ds, idxs = DatasetFormatter().from_toplosses(learn_rel)

Now we will use the widget to delete or move images. Flag photos for deletion by clicking 'Delete' or move them by using the dropdown menu. Then click 'Next Batch' to delete flagged photos and keep the rest in that row. `ImageCleaner` will show you a new row of images until there are no more to show. 

When you change your dataset, `ImageCleaner` will save the new dataset in a 'cleaned.csv' file in the same file where you have your notebook. 

In [16]:
ImageCleaner(ds, idxs, path)

Button(button_style='primary', description='Next Batch', layout=Layout(width='auto'), style=ButtonStyle())

You can also find duplicates in your dataset and delete them! First we need to load our data from the csv and create a new `.Learner` object. We will only use 10000 examples in the dataset since examining the whole dataset would take a long time. To compute similarities for the whole data just delete the line with `df.head(10000)`.

In [9]:
df = pd.read_csv(path/'cleaned.csv', header='infer')

In [10]:
df = df.head(10000)

In [11]:
# We create a databunch from our csv. We include the data in the training set and we don't use a validation set (DatasetFormatter uses only the training set)
np.random.seed(42)
db = (ImageItemList.from_df(df, path)
                   .no_split()
                   .label_from_df()
                   .databunch())

In [12]:
learn_dup = create_cnn(db, models.resnet34, metrics=error_rate)
learn_dup.load('stage-1');

We will first get the indexes for the most similar images and ImageCleaner will use them to find the potential duplicates. This step is quite memory intensive, so you might want to do monitor your RAM to see if your machine can handle it. It also takes quite a lot of time, if you want to make it faster you can edit the `pool_dim` parameter to make the feature vectors smaller but this will decrease the quality of the similarities.

In [13]:
ds, idxs = DatasetFormatter().from_similars(learn_dup, pool_dim=1)

Getting activations...


Computing similarities...


In [17]:
idxs

[8593,
 8477,
 7708,
 7617,
 7850,
 7357,
 500,
 70,
 7799,
 7010,
 2681,
 2663,
 2880,
 2813,
 4738,
 4340,
 3509,
 3256,
 8249,
 8090,
 8920,
 8010,
 4755,
 530,
 6955,
 6872,
 8806,
 8083,
 9817,
 8707,
 2630,
 2628,
 7703,
 7041,
 7858,
 7306,
 8676,
 8627,
 6780,
 6065,
 8639,
 8249,
 590,
 54,
 8848,
 8166,
 6619,
 6136,
 7450,
 530,
 9588,
 8083,
 2451,
 2228,
 8583,
 8122,
 3677,
 3366,
 8481,
 2870,
 7976,
 7243,
 6799,
 6613,
 9189,
 6390,
 7513,
 7357,
 4141,
 4043,
 6956,
 4993,
 7880,
 7023,
 9817,
 8530,
 9657,
 7330,
 8885,
 8058,
 8760,
 8104,
 9319,
 8194,
 9817,
 8441,
 9243,
 7041,
 6037,
 356,
 9509,
 9088,
 7938,
 7740,
 9521,
 8083,
 7128,
 6125,
 6770,
 6670,
 9817,
 8806,
 9056,
 2920,
 8535,
 8459,
 8920,
 8249,
 4423,
 4043,
 6546,
 6237,
 9817,
 8083,
 8249,
 8010,
 6127,
 5962,
 2848,
 2223,
 2936,
 2251,
 9243,
 2194,
 4784,
 2538,
 7880,
 7770,
 6448,
 6096,
 9989,
 6914,
 8303,
 2216,
 6725,
 6419,
 7853,
 7435,
 9989,
 6918,
 9173,
 530,
 6579,
 6138,
 9

Take a look at the images in pairs and delete the ones you don't want to see anymore, until you feel that images don't look alike anymore. Remember that if you want to rerun the widget you need to recreate the `ImageDataBunch` object, loading the data from `cleaned.csv`.

In [20]:
ImageCleaner(ds, idxs, path, duplicates=True)

Button(button_style='primary', description='Next Batch', layout=Layout(width='auto'), style=ButtonStyle())

Turns out there is quite a number of duplicates in CIFAR!

## Train with new dataset

Now you are ready to work in your real training with a clean dataset! To use the new dataset you must load the labels and files from csv into your ImageDataBunch object.

In [86]:
# We create a databunch from our csv. We include the data in the training set and we don't use a validation set (DatasetFormatter uses only the training set)
np.random.seed(42)
db = (ImageItemList.from_df(df, path)
                   .no_split()
                   .label_from_df()
                   .databunch())

In [87]:
learn = create_cnn(db, models.resnet34, metrics=error_rate)

In [45]:
learn.fit_one_cycle(4)

epoch,train_loss,valid_loss,error_rate
1,2.697477,1.928366,0.726316
2,2.147982,1.205164,0.421053
3,1.731222,1.010826,0.294737
4,1.483092,0.964751,0.326316
